In [1]:
from fastai.tabular.all import *
from transfertab.utils import *
from transfertab.transfer import *
from transfertab.extract import *
from fastcore.xtras import *
from fastai.learner import *
import torch
import numpy as np
from fastai.interpret import *
import wandb
from fastai.callback.wandb import *

In [2]:
df = pd.read_csv("../data/datasets/online_shoppers_intention.csv")

In [3]:
df.shape

(12330, 18)

In [4]:
df.shape[0]*0.6

7398.0

In [5]:
dfA = df.iloc[0:df.shape[0]*0.6]
dfB = df.iloc[df.shape[0]*0.6:-1]

In [6]:
dfA = df.iloc[0:7398]
dfB = df.iloc[7398:-1]

In [7]:
dfA.columns

Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend', 'Revenue'],
      dtype='object')

In [8]:
dfA.head()

   Administrative  Administrative_Duration  Informational  \
0               0                      0.0              0   
1               0                      0.0              0   
2               0                      0.0              0   
3               0                      0.0              0   
4               0                      0.0              0   

   Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                     0.0               1                 0.000000   
1                     0.0               2                64.000000   
2                     0.0               1                 0.000000   
3                     0.0               2                 2.666667   
4                     0.0              10               627.500000   

   BounceRates  ExitRates  PageValues  SpecialDay Month  OperatingSystems  \
0         0.20       0.20         0.0         0.0   Feb                 1   
1         0.00       0.10         0.0         0.0   Feb   

In [9]:
dfA.ProductRelated.unique()

array([  1,   2,  10,  19,   0,   3,  16,   7,   6,  23,  13,  20,   8,
         5,  32,   4,  45,  14,  52,   9,  46,  15,  22,  11,  12,  36,
        42,  27,  90,  18,  38,  17, 128,  25,  30,  21,  51,  26,  28,
        31,  24,  50,  96,  49,  68,  98,  67,  55,  35,  37,  29,  34,
        71,  63,  87,  40,  33,  54,  64,  75,  39, 111,  81,  61,  47,
        44,  88, 149,  41,  79,  66,  43, 258,  80,  62,  83, 173,  48,
        58,  57,  56,  69,  82,  59, 109, 287,  53,  84,  78, 137, 113,
        89,  65,  60, 104, 129,  77,  74,  93,  76,  72, 194, 140, 110,
       132, 115,  73, 328, 160,  86, 150,  95, 130, 151, 117, 124, 127,
       125, 116, 105,  92, 157, 154, 220, 187, 112, 131, 159,  94, 204,
       142, 206, 102, 313, 145,  85,  97, 198, 181, 126, 106, 101, 108,
       119,  70, 122,  91, 276, 100, 291, 114, 172, 217, 141, 133, 156,
       136, 180, 135, 195,  99, 362, 179, 118, 175, 148, 440, 103, 178,
       184, 705, 134, 176, 146, 189, 120, 193, 222, 121, 107, 30

In [10]:
len(dfA.ProductRelated.unique())

221

In [11]:
catcolsA = ['Administrative', 'Informational',
       'ProductRelated',
       'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend']
contcolsA = ['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues']
target='Revenue'

In [12]:
splitsA = RandomSplitter(valid_pct=0.)(range_of(dfA))

In [13]:
splitsA

((#7398) [6251,6039,1475,2296,525,724,673,5259,2459,2786...], (#0) [])

In [14]:
splitsA = RandomSplitter(valid_pct=0.)(range_of(dfA))
splitA

In [15]:
splitsA = RandomSplitter(valid_pct=0.)(range_of(dfA))
splitsA

((#7398) [1665,5473,5745,2851,1010,790,3673,3021,3687,5802...], (#0) [])

In [16]:
splitsA = RandomSplitter(valid_pct=0.)(range_of(dfA))
toA = TabularPandas(dfA, procs=[Categorify, FillMissing, Normalize],
                   cat_names=catcolsA,
                   cont_names=contcolsA,
                   y_names=target,
                   splits=splitsA)
dlsA = toA.dataloaders(bs=512)
rocaucbinA = RocAucBinary()
learnA = tabular_learner(dlsA, metrics=[rocaucbinA, accuracy])

In [17]:
wandb.init(
    project='Experiments', 
    save_code=True, 
    group='Exp5c', 
    job_type='pretrain', 
    tags=['base', 'shopping', 'revenue', 'modelA'], 
    name='modelA training',
    notes="Training modelA from which we'll take embeddings",
    reinit=True,
    dir='/Users/manikyabardhan/Documents/github/transfertab/experiments/results/exp5c',
    entity='transfertab'
)

In [18]:
cbs=[WandbCallback(log="all", dataset_name="online_shoppers_intention (first .6)", n_preds=128, seed=1), SaveModelCallback()]
learnA.fit_one_cycle(30, cbs=cbs)

epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


...

epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


In [19]:
cbs=[WandbCallback(log="all", dataset_name="online_shoppers_intention (first .6)", n_preds=0, seed=1), SaveModelCallback()]
learnA.fit_one_cycle(30, cbs=cbs)

epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


...

epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


In [20]:
cbs=[WandbCallback(log_preds=False, log="all", dataset_name="online_shoppers_intention (first .6)", n_preds=0, seed=1), SaveModelCallback()]
learnA.fit_one_cycle(30, cbs=cbs)

epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


...

epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


In [21]:
splitsB = RandomSplitter(valid_pct=0.)(range_of(dfB))
toB = TabularPandas(dfB, procs=[Categorify, FillMissing, Normalize],
                   cat_names = catcolsA,
                   cont_names = contcolsA,
                   y_names=target,
                   splits=splitsB)
dlsB = toB.dataloaders(bs=512)

In [22]:
dlsB.train_ds

       Administrative  Administrative_Duration  Informational  \
8566                1                -0.463323              1   
7593                1                -0.463323              1   
9118                1                -0.463323              2   
10629               1                -0.463323              1   
7548                1                -0.463323              1   
...               ...                      ...            ...   
7763               16                 0.973577              1   
11847               5                 0.312918              1   
11001               4                 0.116448              1   
8981                1                -0.463323              1   
7525                2                -0.436339              2   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
8566                -0.256043              22                -0.430682   
7593                -0.256043               7                -0.137928 

In [23]:
dlsB.train

In [24]:
splitsA = RandomSplitter(valid_pct=0.)(range_of(dfA))
toA = TabularPandas(dfA, procs=[Categorify, FillMissing, Normalize],
                   cat_names=catcolsA,
                   cont_names=contcolsA,
                   y_names=target,
                   splits=splitsA)
dlsA = toA.dataloaders(bs=512)
rocaucbinA = RocAucBinary()
learnA = tabular_learner(dlsA, metrics=[rocaucbinA, accuracy])

In [25]:
cbs=[WandbCallback(valid_dl = dlsB.train,log_preds=False, log="all", dataset_name="online_shoppers_intention (first .6)", n_preds=0, seed=1), SaveModelCallback()]
learnA.fit_one_cycle(30, cbs=cbs)

epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


In [26]:
splitsA = RandomSplitter(valid_pct=0.1)(range_of(dfA))
splitsA

((#6659) [1937,542,2390,4617,2200,6560,3901,1013,5746,4067...],
 (#739) [3489,5924,274,5685,5393,589,5247,3033,4472,7113...])

In [27]:
splitsA = RandomSplitter(valid_pct=0.01)(range_of(dfA))
splitsA

((#7325) [462,7362,7346,3655,23,7313,3914,2245,3813,6703...],
 (#73) [3480,4600,5924,3775,4288,2569,5896,6905,4041,1665...])

In [28]:
toA = TabularPandas(dfA, procs=[Categorify, FillMissing, Normalize],
                   cat_names=catcolsA,
                   cont_names=contcolsA,
                   y_names=target,
                   splits=splitsA)
dlsA = toA.dataloaders(bs=512)
rocaucbinA = RocAucBinary()
learnA = tabular_learner(dlsA, metrics=[rocaucbinA, accuracy])

In [29]:
splitsB = RandomSplitter(valid_pct=0.01)(range_of(dfB))
toB = TabularPandas(dfB, procs=[Categorify, FillMissing, Normalize],
                   cat_names = catcolsA,
                   cont_names = contcolsA,
                   y_names=target,
                   splits=splitsB)
dlsB = toB.dataloaders(bs=512)

In [30]:
cbs=[WandbCallback(valid_dl = dlsB.train,log_preds=False, log="all", dataset_name="online_shoppers_intention (first .6)", n_preds=0, seed=1), SaveModelCallback()]
learnA.fit_one_cycle(30, cbs=cbs)

epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time


...

epoch,train_loss,valid_loss,roc_auc_score,accuracy,time
0,0.745902,0.756442,0.873264,0.136986,00:00
1,0.688141,0.724496,0.946181,0.328767,00:00
2,0.638368,0.669098,0.942708,0.890411,00:01
3,0.598680,0.598725,0.944444,0.931507,00:01
4,0.562174,0.548346,0.947917,0.904110,00:01
5,0.529628,0.511072,0.951389,0.890411,00:01
6,0.496139,0.441277,0.953125,0.890411,00:00
7,0.458926,0.378440,0.956597,0.917808,00:00
8,0.421314,0.289449,0.968750,0.917808,00:00
9,0.383581,0.202608,0.973958,0.945205,00:00


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time
0,0.745902,0.756442,0.873264,0.136986,00:00
1,0.688141,0.724496,0.946181,0.328767,00:00
2,0.638368,0.669098,0.942708,0.890411,00:01
3,0.598680,0.598725,0.944444,0.931507,00:01
4,0.562174,0.548346,0.947917,0.904110,00:01
5,0.529628,0.511072,0.951389,0.890411,00:01
6,0.496139,0.441277,0.953125,0.890411,00:00
7,0.458926,0.378440,0.956597,0.917808,00:00
8,0.421314,0.289449,0.968750,0.917808,00:00
9,0.383581,0.202608,0.973958,0.945205,00:00


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time
0,0.745902,0.756442,0.873264,0.136986,00:00
1,0.688141,0.724496,0.946181,0.328767,00:00
2,0.638368,0.669098,0.942708,0.890411,00:01
3,0.598680,0.598725,0.944444,0.931507,00:01
4,0.562174,0.548346,0.947917,0.904110,00:01
5,0.529628,0.511072,0.951389,0.890411,00:01
6,0.496139,0.441277,0.953125,0.890411,00:00
7,0.458926,0.378440,0.956597,0.917808,00:00
8,0.421314,0.289449,0.968750,0.917808,00:00
9,0.383581,0.202608,0.973958,0.945205,00:00


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time
0,0.745902,0.756442,0.873264,0.136986,00:00
1,0.688141,0.724496,0.946181,0.328767,00:00
2,0.638368,0.669098,0.942708,0.890411,00:01
3,0.598680,0.598725,0.944444,0.931507,00:01
4,0.562174,0.548346,0.947917,0.904110,00:01
5,0.529628,0.511072,0.951389,0.890411,00:01
6,0.496139,0.441277,0.953125,0.890411,00:00
7,0.458926,0.378440,0.956597,0.917808,00:00
8,0.421314,0.289449,0.968750,0.917808,00:00
9,0.383581,0.202608,0.973958,0.945205,00:00


epoch,train_loss,valid_loss,roc_auc_score,accuracy,time
0,0.745902,0.756442,0.873264,0.136986,00:00
1,0.688141,0.724496,0.946181,0.328767,00:00
2,0.638368,0.669098,0.942708,0.890411,00:01
3,0.598680,0.598725,0.944444,0.931507,00:01
4,0.562174,0.548346,0.947917,0.904110,00:01
5,0.529628,0.511072,0.951389,0.890411,00:01
6,0.496139,0.441277,0.953125,0.890411,00:00
7,0.458926,0.378440,0.956597,0.917808,00:00
8,0.421314,0.289449,0.968750,0.917808,00:00
9,0.383581,0.202608,0.973958,0.945205,00:00


In [31]:
from fastai.tabular.all import *
from transfertab.utils import *
from transfertab.transfer import *
from transfertab.extract import *
from fastcore.xtras import *
from fastai.learner import *
import torch
import numpy as np
from fastai.interpret import *
import wandb
from fastai.callback.wandb import *

In [32]:
df = pd.read_csv("../data/datasets/online_shoppers_intention.csv")

In [33]:
df.shape[0]*0.6

7398.0

In [34]:
dfA = df.iloc[0:7398]
dfB = df.iloc[7398:-1]

In [35]:
dfA.columns

Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend', 'Revenue'],
      dtype='object')

In [36]:
dfA.head()

   Administrative  Administrative_Duration  Informational  \
0               0                      0.0              0   
1               0                      0.0              0   
2               0                      0.0              0   
3               0                      0.0              0   
4               0                      0.0              0   

   Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                     0.0               1                 0.000000   
1                     0.0               2                64.000000   
2                     0.0               1                 0.000000   
3                     0.0               2                 2.666667   
4                     0.0              10               627.500000   

   BounceRates  ExitRates  PageValues  SpecialDay Month  OperatingSystems  \
0         0.20       0.20         0.0         0.0   Feb                 1   
1         0.00       0.10         0.0         0.0   Feb   

In [37]:
len(dfA.ProductRelated.unique())

221

In [38]:
catcolsA = ['Administrative', 'Informational',
       'ProductRelated',
       'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend']
contcolsA = ['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues']
target='Revenue'

In [39]:
splitsA = RandomSplitter(valid_pct=0.01)(range_of(dfA))
splitsA

((#7325) [5295,3011,3944,4591,613,378,1347,6118,5232,3122...],
 (#73) [6626,2765,4524,4360,6964,6374,7275,1478,628,5787...])

In [40]:
toA = TabularPandas(dfA, procs=[Categorify, FillMissing, Normalize],
                   cat_names=catcolsA,
                   cont_names=contcolsA,
                   y_names=target,
                   splits=splitsA)
dlsA = toA.dataloaders(bs=512)
rocaucbinA = RocAucBinary()
learnA = tabular_learner(dlsA, metrics=[rocaucbinA, accuracy])

In [41]:
splitsB = RandomSplitter(valid_pct=0.01)(range_of(dfB))
toB = TabularPandas(dfB, procs=[Categorify, FillMissing, Normalize],
                   cat_names = catcolsA,
                   cont_names = contcolsA,
                   y_names=target,
                   splits=splitsB)
dlsB = toB.dataloaders(bs=512)

In [42]:
wandb.init(
    project='Experiments', 
    save_code=True, 
    group='Exp5c', 
    job_type='pretrain', 
    tags=['base', 'shopping', 'revenue', 'modelA'], 
    name='modelA training',
    notes="Training modelA from which we'll take embeddings",
    reinit=True,
    dir='/Users/manikyabardhan/Documents/github/transfertab/experiments/results/exp5c',
    entity='transfertab'
)